## Tectonics and Landscape Evolution Project



In [6]:
from landlab import RasterModelGrid 

import pandas as pd
import numpy as np
from FieldWork_class import FieldWork

#Import you field work data
data = pd.read_csv('/Users/poliveira/Projects_PhD/tect_land_project/field_work_tatun_park.csv')
data

#Calculate parameters for your model
fw = FieldWork(data)
result = fw.calculate_dip_strike()

#Set the size of your area
w = 1000
l  = 1000
cell_resolution = 100

result_2 = fw.calculate_grid_size(w,l,cell_resolution) 

my_grid = RasterModelGrid(rows,cols,cell_resolution)




The average strike is 95.0
The average dip is 8.0
Your area have 1000x1000, the grid has 10.0 rows and 10.0 columns


NameError: name 'rows' is not defined